# AstroPopoteAI v0.1.1

# Steps to run
1. Runtime -> Run All  OR  Ctrl-f9 OR Click the ⏵ button on the left for each of the cells below
2. Enjoy! (Open the url at the last cell)
3. Give the [repo](https://github.com/nicastel/AstroPopoteAI) a ⭐ so other people can find out about it.

In [ ]:
# @title Download AstroPopoteAI and dependencies
%cd /content/
!wget https://github.com/nicastel/AstroPopoteAI/archive/refs/tags/0.1.1.zip
!mv 0.1.1.zip AstroPopoteAI.zip
!unzip -o AstroPopoteAI.zip
!rm -rf AstroPopoteAI
!mv AstroPopoteAI-0.1.1 AstroPopoteAI
#dependencies
%cd AstroPopoteAI
!pip3 install -r requirements.txt # onnx cpu, but we can use colabs free gpu with:
#siril
!wget https://gitlab.com/free-astro/pysiril/uploads/8224707c29669f255ad43da3b93bc5ec/pysiril-0.0.15-py3-none-any.whl
!pip3 install pysiril-0.0.15-py3-none-any.whl
!add-apt-repository -y ppa:lock042/siril
!apt-get install siril
!wget https://free-astro.org/download/kstars-siril-catalogues/namedstars.dat.xz
!unxz namedstars.dat.xz
!wget https://free-astro.org/download/kstars-siril-catalogues/unnamedstars.dat.xz
!unxz unnamedstars.dat.xz
!wget https://free-astro.org/download/kstars-siril-catalogues/deepstars.dat.xz
!unxz deepstars.dat.xz
!wget https://free-astro.org/download/kstars-siril-catalogues/USNO-NOMAD-1e8.dat.xz
!unxz USNO-NOMAD-1e8.dat.xz
# astap
!wget https://github.com/nicastel/AstroPopoteAI/releases/download/astap/astap_command-line_version_Linux_amd64.zip
!unzip astap_command-line_version_Linux_amd64.zip
!wget -O d20_star_database.zip "https://drive.usercontent.google.com/download?id=1aCAKK0tB6eCNrzqPvwfCq-vg70ik0Ug4&export=download&authuser=0&confirm=t&uuid=a4415b42-70d3-4bd4-8025-7889ce24e518&at=APZUnTWIGJsa5pA0mQlPiX83UAgi%3A1713168442602"
!unzip d20_star_database.zip
# graxpert
!wget https://github.com/Steffenhir/GraXpert/archive/refs/tags/3.0.2.zip
!mv 3.0.2.zip GraXpert-3.0.2.zip
!unzip GraXpert-3.0.2.zip
!cp s3_secrets.py GraXpert-3.0.2/graxpert/
# starnet
!wget https://github.com/nicastel/starnet/releases/download/starnetv1/starnet_weights2.zip
!unzip starnet_weights2.zip
!chmod +x run_starnet.sh

/content
--2024-08-05 15:42:11--  https://github.com/nicastel/AstroPopoteAI/archive/refs/tags/0.1.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nicastel/AstroPopoteAI/zip/refs/tags/0.1 [following]
--2024-08-05 15:42:11--  https://codeload.github.com/nicastel/AstroPopoteAI/zip/refs/tags/0.1
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘0.1.zip’

0.1.zip                 [ <=>                ]  26.12K  --.-KB/s    in 0.002s  

2024-08-05 15:42:12 (15.7 MB/s) - ‘0.1.zip’ saved [26745]

Archive:  AstroPopoteAI.zip
8cf30ce398dad5b0826ab6c8e44a625e0d4d56ca
   creating: AstroPopoteAI-0.1/
  inflating: AstroPopoteAI-0.1/

In [ ]:
#@title Define tunnel functions
import atexit, os, platform, re, subprocess, requests, stat, secrets, time
from pathlib import Path
from typing import List

VERSION = "0.2"
CURRENT_TUNNELS: List["Tunnel"] = []
GRADIO_API_SERVER = "https://api.gradio.app/v2/tunnel-request"

class Tunnel:
    def __init__(self, remote_host, remote_port, local_host, local_port, share_token):
        self.proc = None
        self.url = None
        self.remote_host = remote_host
        self.remote_port = remote_port
        self.local_host = local_host
        self.local_port = local_port
        self.share_token = share_token

    @staticmethod
    def download_binary():
        machine = platform.machine()
        if machine == "x86_64":
            machine = "amd64"

        binary_name = f"frpc_{platform.system().lower()}_{machine.lower()}"
        #binary_path = str(Path(__file__).parent / binary_name) + f"_v{VERSION}"

        binary_path = f"/content/AstroPopoteAI/{binary_name}_v{VERSION}"

        extension = ".exe" if os.name == "nt" else ""

        if not Path(binary_path).exists():
            binary_url = f"https://cdn-media.huggingface.co/frpc-gradio-{VERSION}/{binary_name}{extension}"
            resp = requests.get(binary_url)

            if resp.status_code == 403:
                raise OSError(
                    f"Cannot set up a share link as this platform is incompatible. Please "
                    f"create a GitHub issue with information about your platform: {platform.uname()}"
                )

            resp.raise_for_status()

            with open(binary_path, "wb") as file:
                file.write(resp.content)
            st = os.stat(binary_path)
            os.chmod(binary_path, st.st_mode | stat.S_IEXEC)

        return binary_path

    def start_tunnel(self) -> str:
        binary_path = self.download_binary()
        self.url = self._start_tunnel(binary_path)
        return self.url

    def kill(self):
        if self.proc is not None:
            print(f"Killing tunnel {self.local_host}:{self.local_port} <> {self.url}")
            self.proc.terminate()
            self.proc = None

    def _start_tunnel(self, binary: str) -> str:
        CURRENT_TUNNELS.append(self)
        command = [binary, "http", "-n", self.share_token, "-l", str(self.local_port), "-i", self.local_host, "--uc", "--sd", "random", "--ue", "--server_addr", f"{self.remote_host}:{self.remote_port}", "--disable_log_color"]
        self.proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        atexit.register(self.kill)
        url = ""
        while url == "":
            if self.proc.stdout is None:
                continue
            line = self.proc.stdout.readline()
            line = line.decode("utf-8")
            if "start proxy success" in line:
                result = re.search("start proxy success: (.+)\n", line)
                if result is None:
                    raise ValueError("Could not create share URL")
                else:
                    url = result.group(1)
        return url

def setup_tunnel(local_host: str, local_port: int, share_token: str) -> str:
    response = requests.get(GRADIO_API_SERVER)
    if response and response.status_code == 200:
        try:
            payload = response.json()[0]
            remote_host, remote_port = payload["host"], int(payload["port"])
            tunnel = Tunnel( remote_host, remote_port, local_host, local_port, share_token )
            address = tunnel.start_tunnel()
            return address
        except Exception as e:
            raise RuntimeError(str(e))
    else:
        raise RuntimeError("Could not get share link from Gradio API Server.")

In [ ]:
# @title Start AstroPopoteAI

%cd /content/AstroPopoteAI/

tunnel_url = setup_tunnel("localhost", 7860, secrets.token_urlsafe(32))
print(f"If everything's working, you can can finally start cooking!\nPlease visit: {tunnel_url} after the final cell starts running\n\nGo here for some general tips & tricks, and remember to give the project a star if you found it useful!\nhttps://github.com/nicastel/AstroPopoteAI")

!streamlit run app.py --server.port 7860

/content/AstroPopoteAI
If everything's working, you can can finally start cooking!
Please visit: https://3e51542b466baf2f3c.gradio.live after the final cell starts running

Go here for some general tips & tricks, and remember to give the project a star if you found it useful!
https://github.com/nicastel/AstroPopoteAI



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:7860
  Network URL: http://172.28.0.12:7860
  External URL: http://35.187.240.131:7860

INFO   : VERSION siril 1.2.2 : 
INFO   : Siril is compatible with pysiril
INFO   : Initialisation pySiril V0.0.15 : OK
INFO   : First step: Starting 'Siril -p' ....
INFO   : ThreadSiril is started
INFO   : run : /usr/bin/siril-cli -p
INFO   : waiting: 
7s 
INFO   : Second step: Starting pipe reader ....
INFO   : ThreadReader is started
INFO   : Third step: Starting pipe writer ....
INFO   : PipeWriter is started
INFO   : Open()
: ready
> requires 1.2.2
: log: Running command: requires
> [status: suc